In [64]:
# feature_set = ['Slope_Gamma_channel77', 'Slope_Gamma_channel61', 'Slope_Delta_channel105', 'Slope_Gamma_channel65', 
#                'Slope_Delta_channel95', 'Slope_Delta_channel109', 'Slope_Gamma_channel60', 'Slope_Gamma_channel26', 
#                'Slope_Delta_channel43', 'Slope_Delta_channel40', 
#     'bdi_binary']

## 1. Odds Ratio for all the dataset without train/test splits + p values

Logistic regression on all data set and calculate odd ratio for each MEG feature? when dependent variable is BDI/Apathy.
CI would also be great.

In [67]:
import pandas as pd 
import numpy as np

df_psd = pd.read_csv("data/df_meg_psy_allfeatures.csv")
df_slopes = pd.read_csv("data/slopes_580features.csv")

df_slopes = df_slopes[df_slopes['target']==1].drop(columns='target')

In [69]:
df_psd.head(2)

,PATIENT_ID,MoCA corretto,LEDD Levodopa,LEDD_DA,LEDD totale,AGE,SEX,DD months,mH&Y,UPDRS III,...,Delta_channel114,Theta_channel114,Alpha_channel114,Beta_channel114,Gamma_channel114,Delta_channel115,Theta_channel115,Alpha_channel115,Beta_channel115,Gamma_channel115
0,BNTLGU53_01,20.98,150,0,150,63,0,24,2.0,29,...,40375.942038,24202.162437,37191.128954,7215.472327,1711.428597,48377.700893,28849.230923,43652.571297,8903.325322,2183.211469
1,BRRMRS48_01,14.40,300,0,400,70,1,24,2.0,21,...,12839.523331,5403.541702,7058.583578,2759.861741,1139.080707,20525.446929,8569.110052,10912.941664,4407.627448,1845.709506


In [71]:
df_slopes.head(2)

,Slope_Delta_channel1,Slope_Theta_channel1,Slope_Alpha_channel1,Slope_Beta_channel1,Slope_Gamma_channel1,Slope_Delta_channel2,Slope_Theta_channel2,Slope_Alpha_channel2,Slope_Beta_channel2,Slope_Gamma_channel2,...,Slope_Delta_channel115,Slope_Theta_channel115,Slope_Alpha_channel115,Slope_Beta_channel115,Slope_Gamma_channel115,Slope_Delta_channel116,Slope_Theta_channel116,Slope_Alpha_channel116,Slope_Beta_channel116,Slope_Gamma_channel116
20,-0.655566,0.142126,-0.843033,-0.875152,-10.003454,-0.692597,0.224640,-0.906742,-1.141044,-9.867734,...,-0.686671,0.090868,0.888668,-1.909757,-9.392111,-0.701925,0.042903,0.905722,-1.871405,-9.378056
21,-0.359439,0.367208,-2.740478,-1.716018,-9.740200,-0.326454,0.706182,-2.821907,-1.638603,-9.812921,...,-0.646110,1.135646,-3.535051,-2.086474,-9.405221,-0.588811,1.123969,-3.424744,-2.136706,-9.432555


In [73]:
vector_bdi = df_psd["BDI"]
vector_apathy = df_psd["Apathy"]

In [75]:
thr1 = vector_bdi.quantile(0.5)+0.1  # This gives the median value
thr2 = vector_apathy.quantile(0.5)+0.1  # This gives the median value

bdi_binary = (vector_bdi >= thr1).astype(int)
apathy_binary = (vector_apathy >= thr2).astype(int)

In [77]:
bdi_binary.value_counts()

BDI
0    11
1     9
Name: count, dtype: int64

In [79]:
apathy_binary.value_counts()

Apathy
0    10
1    10
Name: count, dtype: int64

In [81]:
df_slopes = df_slopes.reset_index().drop(columns='index')

In [83]:
df_slopes_apathy = df_slopes.copy(deep=True)
df_slopes_bdi = df_slopes.copy(deep=True)
df_slopes_apathy['apathy_binary'] = apathy_binary
df_slopes_bdi['bdi_binary'] = bdi_binary

df_psd_apathy = df_psd.copy(deep=True)
df_psd_bdi = df_psd.copy(deep=True)
df_psd_apathy['apathy_binary'] = apathy_binary
df_psd_bdi['bdi_binary'] = bdi_binary

In [87]:
print(df_slopes_bdi.shape)
print(df_slopes_apathy.shape)

print(df_psd_apathy.shape)
print(df_psd_bdi.shape)

(20, 581)
(20, 581)
(20, 600)
(20, 600)


In [89]:
# logistic regression 
from sklearn import svm, metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, KFold, train_test_split, cross_val_predict, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier

import warnings
from sklearn.exceptions import ConvergenceWarning

In [90]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.utils import resample


# ML - Slopes

In [94]:
df_psd_apathy = df_psd_apathy.iloc[:, 19:]
df_psd_bdi = df_psd_bdi.iloc[:, 19:]

In [98]:
df_current = df_slopes_apathy.copy(deep=True)
X = df_current.drop(columns='apathy_binary')
y = df_current['apathy_binary']

In [102]:
df_current.shape

(20, 581)

In [105]:
df_current.head(2)

,Slope_Delta_channel1,Slope_Theta_channel1,Slope_Alpha_channel1,Slope_Beta_channel1,Slope_Gamma_channel1,Slope_Delta_channel2,Slope_Theta_channel2,Slope_Alpha_channel2,Slope_Beta_channel2,Slope_Gamma_channel2,...,Slope_Theta_channel115,Slope_Alpha_channel115,Slope_Beta_channel115,Slope_Gamma_channel115,Slope_Delta_channel116,Slope_Theta_channel116,Slope_Alpha_channel116,Slope_Beta_channel116,Slope_Gamma_channel116,apathy_binary
0,-0.655566,0.142126,-0.843033,-0.875152,-10.003454,-0.692597,0.224640,-0.906742,-1.141044,-9.867734,...,0.090868,0.888668,-1.909757,-9.392111,-0.701925,0.042903,0.905722,-1.871405,-9.378056,0
1,-0.359439,0.367208,-2.740478,-1.716018,-9.740200,-0.326454,0.706182,-2.821907,-1.638603,-9.812921,...,1.135646,-3.535051,-2.086474,-9.405221,-0.588811,1.123969,-3.424744,-2.136706,-9.432555,0


# Test with researchpy

In [110]:
!pip install researchpy

In [112]:
import pandas as pd
import researchpy as rp
import statsmodels.api as sm


df = pd.read_stata("https://stats.idre.ucla.edu//stat//stata//dae//binary.dta")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   admit   400 non-null    float32
 1   gre     400 non-null    float32
 2   gpa     400 non-null    float32
 3   rank    400 non-null    float32
dtypes: float32(4)
memory usage: 6.4 KB


In [116]:
rp.summary_cat(df[['admit', 'rank']])

,Variable,Outcome,Count,Percent
0,admit,0.0,273,68.25
1,,1.0,127,31.75
2,rank,2.0,151,37.75
3,,3.0,121,30.25
4,,4.0,67,16.75
5,,1.0,61,15.25


In [120]:
rp.summary_cont(df[['gre', 'gpa']])

/opt/anaconda3/lib/python3.12/site-packages/researchpy/summary.py:102: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'gre' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  table_a.iloc[0,0] = ix
/opt/anaconda3/lib/python3.12/site-packages/researchpy/summary.py:102: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'gpa' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  table_a.iloc[0,0] = ix


,Variable,N,Mean,SD,SE,95% Conf.,Interval
0,gre,400.0,587.7000,115.5167,5.7758,576.3452,599.0549
1,gpa,400.0,3.3899,0.3806,0.0190,3.3525,3.4273


In [ ]:
import statsmodels.formula.api as smf

# https://www.pythonfordatascience.org/logistic-regression-python/

model = smf.logit("admit ~ gre + gpa + C(rank)", data=df).fit()
#model = smf.logit("admit ~ . + C(rank)", data=df).fit()
model.summary()

model_odds = pd.DataFrame(np.exp(model.params), columns= ['OR'])
model_odds['z-value']= model.pvalues
model_odds[['2.5%', '97.5%']] = np.exp(model.conf_int())

model_odds

In [299]:
df_current = df_slopes_apathy.copy(deep=True)
X = df_current.drop(columns='apathy_binary')
y = df_current['apathy_binary']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

model = LogisticRegression()
model.fit(X_scaled, y)

odds_ratios = pd.DataFrame(columns=['odds_ratios', 'log_odds', 'p_val'])

odds_ratios = np.exp(model.coef_[0])
odds_ratios_series = pd.Series(odds_ratios, index=X.columns)

In [301]:
odds_ratios_series

Slope_Delta_channel1      1.035422
Slope_Theta_channel1      0.957715
Slope_Alpha_channel1      1.005977
Slope_Beta_channel1       1.096005
Slope_Gamma_channel1      1.055023
                            ...   
Slope_Delta_channel116    0.887202
Slope_Theta_channel116    0.962903
Slope_Alpha_channel116    1.030569
Slope_Beta_channel116     0.994138
Slope_Gamma_channel116    0.944910
Length: 580, dtype: float64

# Multicollinearity 

# Previous

In [171]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

model = LogisticRegression()

# Store accuracy scores for each bootstrap iteration
accuracy_scores = []

# Perform 20 bootstrap repetitions
for i in range(1000):
    # Create a bootstrap sample with replacement
    X_resampled, y_resampled = resample(X_scaled, y, replace=True, n_samples=len(y), random_state=i)
    
    # Split the resampled data into train and test sets (75% train, 25% test)
    split_index = int(0.75 * len(y_resampled))
    X_train, X_test = X_resampled[:split_index], X_resampled[split_index:]
    y_train, y_test = y_resampled[:split_index], y_resampled[split_index:]
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Predict on the test set
    y_pred = model.predict(X_test)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)

# Print the accuracy scores for each bootstrap iteration
print("PSD, Apathy")
#print("Accuracy Scores for each Bootstrap Iteration:", accuracy_scores)
print("Mean Accuracy Score:", np.mean(accuracy_scores))
print("Mean Std Score:", np.std(accuracy_scores))

PSD, Apathy
Mean Accuracy Score: 0.766
Mean Std Score: 0.19950939827486824


In [193]:
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

model = LogisticRegression()

# Store accuracy scores for each bootstrap iteration
accuracy_scores = []

# Perform 1000 bootstrap repetitions
for i in range(1000):
    # Create a bootstrap sample with replacement
    X_resampled, y_resampled = resample(X_scaled, y, replace=True, n_samples=len(y), random_state=i)
    
    # Split the resampled data into train and test sets (75% train, 25% test)
    split_index = int(0.75 * len(y_resampled))
    X_train, X_test = X_resampled[:split_index], X_resampled[split_index:]
    y_train, y_test = y_resampled[:split_index], y_resampled[split_index:]
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Predict on the test set
    y_pred = model.predict(X_test)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)

# Calculate the odds ratios for each feature
odds_ratios = np.exp(model.coef_[0])
odds_ratios_series = pd.Series(odds_ratios, index=X.columns)

# Print the accuracy scores and odds ratios
print("PSD, BDI")
print("Mean Accuracy Score:", np.mean(accuracy_scores))
print("Mean Std Score:", np.std(accuracy_scores))

# Print the odds ratios for each feature
# print("Odds Ratios for each feature:")
# for i, feature_name in enumerate(X.columns):  # Assuming X is a DataFrame with named columns
#     print(f"{feature_name}: {odds_ratios[i]}")

PSD, BDI
Mean Accuracy Score: 0.7218
Mean Std Score: 0.21015413391127952


In [194]:
np.log(odds_ratios_series).sort_values(ascending=False)[:20]

Slope_Delta_channel30    0.106188
Slope_Gamma_channel73    0.090988
Slope_Gamma_channel72    0.087420
Slope_Delta_channel31    0.082215
Slope_Theta_channel44    0.081484
Slope_Beta_channel75     0.077656
Slope_Beta_channel36     0.077349
Slope_Theta_channel42    0.074124
Slope_Theta_channel43    0.071323
Slope_Delta_channel32    0.070933
Slope_Delta_channel19    0.070524
Slope_Delta_channel29    0.070411
Slope_Gamma_channel82    0.069370
Slope_Beta_channel11     0.069163
Slope_Theta_channel22    0.068673
Slope_Beta_channel83     0.065963
Slope_Delta_channel24    0.065131
Slope_Theta_channel4     0.064999
Slope_Beta_channel2      0.063554
Slope_Gamma_channel78    0.062979
dtype: float64

# Final 28.10

In [320]:
vector_bdi = df_psd["BDI"]
vector_apathy = df_psd["Apathy"]

thr1 = vector_bdi.quantile(0.5)+0.1  # This gives the median value
thr2 = vector_apathy.quantile(0.5)+0.1  # This gives the median value

bdi_binary = (vector_bdi >= thr1).astype(int)
apathy_binary = (vector_apathy >= thr2).astype(int)

df_slopes = df_slopes.reset_index().drop(columns='index')

df_slopes_apathy = df_slopes.copy(deep=True)
df_slopes_bdi = df_slopes.copy(deep=True)
df_slopes_apathy['apathy_binary'] = apathy_binary
df_slopes_bdi['bdi_binary'] = bdi_binary

df_psd_apathy = df_psd.copy(deep=True)
df_psd_bdi = df_psd.copy(deep=True)
df_psd_apathy['apathy_binary'] = apathy_binary
df_psd_bdi['bdi_binary'] = bdi_binary

print(df_slopes_bdi.shape)
print(df_slopes_apathy.shape)

print(df_psd_apathy.shape)
print(df_psd_bdi.shape)

(20, 581)
(20, 581)
(20, 600)
(20, 600)


In [329]:
def calc_odds_ratio(df=df_slopes_apathy, target_var='apathy_binary'):
    df_current = df.copy(deep=True)
    X = df_current.drop(columns=target_var)
    y = df_current[target_var]
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    model = LogisticRegression()
    model.fit(X_scaled, y)
    
    odds_ratios = pd.DataFrame(columns=['odds_ratios', 'log_odds'])
    
    odds_ratios['odds_ratios'] = np.exp(model.coef_[0])
    odds_ratios['log_odds'] = model.coef_[0]
    odds_ratios.index = X.columns
    return odds_ratios

In [363]:
odds_slopes_apathy = calc_odds_ratio(df_slopes_apathy, 'apathy_binary')
odds_slopes_bdi = calc_odds_ratio(df_slopes_bdi, 'bdi_binary')

odds_powers_apathy = calc_odds_ratio(df_psd_apathy.iloc[:, 19:], 'apathy_binary')
odds_powers_bdi = calc_odds_ratio(df_psd_bdi.iloc[:, 19:], 'bdi_binary')

In [375]:
odds_slopes_apathy = odds_slopes_apathy.sort_values(by='odds_ratios', ascending=False)
odds_slopes_bdi = odds_slopes_bdi.sort_values(by='odds_ratios', ascending=False)
odds_powers_apathy = odds_powers_apathy.sort_values(by='odds_ratios', ascending=False)
odds_powers_bdi = odds_powers_bdi.sort_values(by='odds_ratios', ascending=False)

In [377]:
odds_slopes_apathy.to_csv("output/odds_slopes_apathy.csv", index=True)
odds_slopes_bdi.to_csv("output/odds_slopes_bdi.csv", index=True)
odds_powers_apathy.to_csv("output/odds_powers_apathy.csv", index=True)
odds_powers_bdi.to_csv("output/odds_powers_bdi.csv", index=True)

In [379]:
pd.read_csv("output/odds_slopes_apathy.csv", index_col=0)

,odds_ratios,log_odds
Slope_Delta_channel30,1.281330,0.247898
Slope_Delta_channel31,1.270593,0.239484
Slope_Delta_channel83,1.232109,0.208727
Slope_Delta_channel19,1.177954,0.163779
Slope_Delta_channel32,1.174441,0.160793
...,...,...
Slope_Delta_channel64,0.868872,-0.140560
Slope_Delta_channel17,0.868160,-0.141379
Slope_Delta_channel21,0.865483,-0.144468
Slope_Delta_channel105,0.864826,-0.145227
